In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [24]:
!ls

saved_models  test_set	tflite_models  training_set


In [10]:
cd gdrive/My Drive/Datasets/cat_vs_dog/

/content/gdrive/My Drive/Datasets/cat_vs_dog


In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop

In [6]:
classifier = Sequential()

classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 21, 21, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 32)          9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

valid_gen = test_datagen.flow_from_directory('test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')


Found 8009 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [13]:
# Quantization aware training
sess = tf.keras.backend.get_session()
tf.contrib.quantize.create_training_graph(sess.graph)
sess.run(tf.global_variables_initializer())


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Inserting fake quant op activation_Mul_quant after loss/dense_1_loss/logistic_loss/mul


In [14]:
classifier.fit_generator(valid_gen,epochs=2)

Epoch 1/2
63/63 [==============================] - 909s 14s/step - loss: 0.7488 - acc: 0.5080
Epoch 2/2
63/63 [==============================] - 7s 109ms/step - loss: 0.6939 - acc: 0.4840


In [17]:
# Print the min max in fakequant
for node in sess.graph.as_graph_def().node:
    if 'weights_quant/AssignMaxLast' in node.name \
        or 'weights_quant/AssignMinLast' in node.name:
        tensor = sess.graph.get_tensor_by_name(node.name + ':0')
        print('{} = {}'.format(node.name, sess.run(tensor)))


Testing ------------
conv2d/weights_quant/AssignMinLast = -0.13767679035663605
conv2d/weights_quant/AssignMaxLast = 0.13782881200313568
conv2d_1/weights_quant/AssignMinLast = -0.1529141664505005
conv2d_1/weights_quant/AssignMaxLast = 0.15601295232772827
conv2d_2/weights_quant/AssignMinLast = -0.1275157332420349
conv2d_2/weights_quant/AssignMaxLast = 0.12115651369094849
dense/weights_quant/AssignMinLast = -0.21155332028865814
dense/weights_quant/AssignMaxLast = 0.22181682288646698
dense_1/weights_quant/AssignMinLast = -0.2071203887462616
dense_1/weights_quant/AssignMaxLast = 0.2278832197189331


In [19]:
classifier.trainable_weights

[<tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 3, 32) dtype=float32>,
 <tf.Variable 'conv2d_1/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 32, 32) dtype=float32>,
 <tf.Variable 'conv2d_2/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'dense/kernel:0' shape=(32, 128) dtype=float32>,
 <tf.Variable 'dense/bias:0' shape=(128,) dtype=float32>,
 <tf.Variable 'dense_1/kernel:0' shape=(128, 1) dtype=float32>,
 <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32>]

In [23]:
CATVSDOG_SAVED_MODEL = "saved_models/catvsdog_quantization_aware"
tf.saved_model.save(classifier, CATVSDOG_SAVED_MODEL)

INFO:tensorflow:Assets written to: saved_models/catvsdog_quantization_aware/assets
